# Numpy Compat

In [1]:
import uarray
import numpy as np

In [2]:
def some_fn(a, b):
    return np.multiply.outer(a, b)[10, 10]

In [3]:
N = 10000

In [4]:
%time some_fn(np.arange(N), np.arange(N))

CPU times: user 290 ms, sys: 452 ms, total: 742 ms
Wall time: 1.37 s


100

In [8]:
%time uarray.optimize(some_fn)(np.arange(N), np.arange(N))

CPU times: user 26.8 ms, sys: 2.41 ms, total: 29.2 ms
Wall time: 28.7 ms


array(100.)